<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/twelvelabs-io/twelvelabs-developer-experience/blob/main/quickstarts/TwelveLabs_Quickstart_Generate.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in  Colab</a>
  </td>
</table>

# Generate text from videos
This guide shows how to utilize the TwelveLabs Python SDK to generate summaries, chapters, and highlights from videos.

Note that the Generate API suite offers three distinct endpoints tailored to meet various requirements. Each endpoint has been designed with specific levels of flexibility and customization to accommodate different needs.

- `/gist`: Generates titles, topics, and hashtags.
- `/summarize`: Generates summaries, chapters, and highlights.
- `/generate`: Generates open-ended texts.

For details, see the [Generate text from video](https://docs.twelvelabs.io/docs/generate-text-from-video) page.

# Prerequisites
Before you begin, ensure the following prerequisites are met:

- [Sign up](https://playground.twelvelabs.io/) for a free account and obtain your API key from the [API Key](https://playground.twelvelabs.io/dashboard/api-key) page. No credit card is required to use the Free plan. This plan allows you to index up to 600 minutes of videos, which is sufficient for a small project. 
- The videos you wish to upload must meet the requirements in the [Prerequisites](https://docs.twelvelabs.io/v1.3/docs/guides/generate-text-from-video/summaries-chapters-and-highlights#prerequisites) section of the **Summaries, chapters, and highlights** page.



# Procedure

## Install the TwelveLabs Python SDK

In [2]:
%pip install -U -q twelvelabs


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import the required packages

In [3]:
import os
from glob import glob
from twelvelabs import TwelveLabs
from twelvelabs.models.task import Task

## Configure your API key


In [ ]:
# For Google Colab, store your API key as a Secret named `TL_API_KEY`. If you don't know how to create a Colab Secret, see https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75.

from google.colab import userdata
TL_API_KEY = userdata.get('TL_API_KEY')

# For other Python environments, you can use environment variables
# TL_API_KEY = os.environ.get('TL_API_KEY')

## Create an index

An index is a basic unit for organizing and storing video data consisting of video embeddings and metadata. Indexes facilitate information retrieval and processing.

When creating a new index, you must specify at least the following information:
- **Name**: Use a brief and descriptive name to facilitate future reference and management.
- **Model configuration**: Provide a list containing the video understanding models and the associated model options you want to enable.

TwelveLabs provides two distinct model types:

- **Embedding models (Marengo)**: This model performs tasks such as search and embedding generation , enabling enhanced video understanding.
- **Generative models (Pegasus)**: This model generates text based on your videos.

Model options determine what information the video understanding model will process:
- **visual**: Analyzes visual content, including actions, objects, events, text (OCR), and brand logos.
- **audio**: Analyzes audio content, including ambient sounds and human speech.

In [5]:
client = TwelveLabs(api_key=TL_API_KEY)

index = client.index.create(
    name="Temp-2",
    models=[
        {
            "name": "pegasus1.2",
            "options": ["visual", "audio"],
        }
    ]
)
print(f"Created index: id={index.id} name={index.name} models={index.models}")


Created index: id=67c83c399d39a8522d595938 name=Temp-2 models=root=[Model(name='pegasus1.2', options=['visual', 'audio'], addons=None, finetuned=False)]


## Upload videos to your index

In [6]:
# The path to the directory containing the videos you wish to upload.
VIDEO_PATH = "funny-cats.mp4"  # In a colab Notebook, just upload the file in the root folder and just give the filename.

video_files = glob(VIDEO_PATH)
if(len(video_files)==0):
  raise Exception("Video file was not provided")
for video_file in video_files:
  print(f"Uploading {video_file}")
  task = client.task.create(index_id=index.id, file=video_file)
  print(f"Created task: id={task.id}")

  # (Optional) Monitor the video indexing process
  # Utility function to print the status of a video indexing task
  def on_task_update(task: Task):
          print(f"  Status={task.status}")
  task.wait_for_done(sleep_interval=50, callback=on_task_update)
  if task.status != "ready":
      raise RuntimeError(f"Indexing failed with status {task.status}")
  print(f"Uploaded {video_file}. The unique identifer of your video is {task.video_id}.")


Uploading funny-cats.mp4
Created task: id=67c83c6dd4e0d626915c1535
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=ready
Uploaded funny-cats.mp4. The unique identifer of your video is 67c83c6d517c1873bbbd6c9f.


## Generate text

The code below iterates through all videos in your index, generating and displaying summaries, chapters, and highlights for each video. Additionally, it creates SEO keywords using a custom prompt.

In [7]:
#Get all videos in an Index
videos = client.index.video.list(index.id)
for video in videos:
    print(f"Generating text for {video.id}")

    res = client.generate.summarize(video_id=video.id, type="summary")
    print(f"Summary: {res.summary}")

    print("Chapters:")
    res = client.generate.summarize(video_id=video.id, type="chapter")
    for chapter in res.chapters:
        print(
            f"  chapter_number={chapter.chapter_number} chapter_title={chapter.chapter_title} chapter_summary={chapter.chapter_summary} start={chapter.start} end={chapter.end}"
        )

    print("Highlights:")
    res = client.generate.summarize(video_id=video.id, type="highlight")
    for highlight in res.highlights:
        print(
            f"  Highlight={highlight.highlight} start={highlight.start} end={highlight.end}"
        )

    res = client.generate.text(video_id=video.id, prompt="Based on this video, I want to generate five keywords for SEO (Search Engine Optimization).")
    print(f"Open-ended Text: {res.data}")

Generating text for 67c83c6d517c1873bbbd6c9f
Summary: The video is a delightful compilation of various feline and some canine antics, capturing the essence of pet behavior in a series of short, engaging clips. It opens with a brown tabby cat standing on a wooden table, dipping its paw into a glass of water, a playful and curious scene that sets the tone for the video. The cat's behavior is followed by a Bengal cat perched inside a bathroom sink, looking around before leaping out onto the floor, showcasing the agility and confidence of these pets.
The video then transitions to a pair of cats sitting at a window sill, one with black markings and the other without, both turning away as if startled by something outside. This is followed by a brief glimpse of two ginger cats running across a road from inside a car, adding a touch of outdoor adventure to the collection. Inside a dimly lit room, a gray cat plays with a dark-colored cat near a cardboard box, their playful interaction disappear